ERROR: Could not find a version that satisfies the requirement aspose-email-python-dotnet (from versions: none)
ERROR: No matching distribution found for aspose-email-python-dotnet

[notice] A new release of pip available: 22.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
# imports

import os
import glob
from dotenv import load_dotenv
import gradio as gr
from langchain.document_loaders import DirectoryLoader, TextLoader, CSVLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain.vectorstores import FAISS
import numpy as np
from sklearn.manifold import TSNE
import plotly.graph_objects as go
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
import time
import random

In [8]:
headers = ['Subject',
 'Body',
 'From: (Name)',
 'From: (Address)',
 'From: (Type)',
 'To: (Name)',
 'To: (Address)',
 'To: (Type)',
 'CC: (Name)',
 'CC: (Address)',
 'CC: (Type)',
 'BCC: (Name)',
 'BCC: (Address)',
 'BCC: (Type)',
 'Billing Information',
 'Categories',
 'Importance',
 'Mileage',
 'Sensitivity']

In [19]:
# Ensure no corrupted data


# Configuration
db_name = "email_db"
file_path = "combined_emails.csv"
openai_key = os.getenv("OPENAI_KEY")


# Rate limiting configuration
MAX_TOKENS_PER_MIN = 1_000_000
EMBEDDING_MODEL_TOKENS = 8191  # Max tokens per request for text-embedding-ada-002

# Load CSV with headers
loader = CSVLoader(
    file_path=file_path,
    csv_args={
        "delimiter": ",",
        "quotechar": '"',
        "fieldnames": headers,  # Include headers here
    },
    encoding="utf-8",
)
data = loader.load()

# Split documents into manageable chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(data)

# Remove any empty or invalid chunks
chunks = [chunk for chunk in chunks if chunk.page_content.strip()]

# Initialize embeddings
embeddings = OpenAIEmbeddings(openai_api_key=openai_key)

# Initialize Chroma vector store
vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=db_name)

# Helper function for batching
def calculate_batch_size(chunks, max_tokens_per_min=MAX_TOKENS_PER_MIN):
    """Yield batches of chunks that stay within the token-per-minute limit."""
    batch = []
    token_count = 0
    for chunk in chunks:
        token_count += len(chunk.page_content)  # Approximate token count
        if token_count > max_tokens_per_min:
            yield batch
            batch = []
            token_count = len(chunk.page_content)
        batch.append(chunk)
    if batch:
        yield batch

# Retry mechanism with exponential backoff
def retry_with_backoff(func, *args, max_retries=5, **kwargs):
    retries = 0
    while retries < max_retries:
        try:
            return func(*args, **kwargs)
        except Exception as e:
            print(f"Retry {retries + 1}/{max_retries} failed: {e}")
            delay = (2 ** retries) + random.uniform(0, 1)
            time.sleep(delay)
            retries += 1
    raise Exception("Max retries reached.")

# Process chunks in batches and apply rate limiting
for batch in calculate_batch_size(chunks):
    try:
        # Generate embeddings for the batch
        documents = [chunk.page_content for chunk in batch]
        metadatas = [chunk.metadata for chunk in batch]  # Ensure metadata exists
        retry_with_backoff(vectorstore.add_texts, documents, metadatas=metadatas)
        # Ensure we don't exceed the token-per-minute rate
        time.sleep(60)  # Wait 1 minute after processing each batch
    except Exception as e:
        print(f"Error processing batch: {e}")


RateLimitError: Error code: 429 - {'error': {'message': 'Request too large for text-embedding-ada-002 in organization org-mksYIsrXkSRYrNxIFytHgf2Z on tokens per min (TPM): Limit 1000000, Requested 1634615. The input or output tokens must be reduced in order to run successfully. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}

RateLimitError: Error code: 429 - {'error': {'message': 'Request too large for text-embedding-ada-002 in organization org-mksYIsrXkSRYrNxIFytHgf2Z on tokens per min (TPM): Limit 1000000, Requested 1634615. The input or output tokens must be reduced in order to run successfully. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}

In [ ]:

    # Step 3: Fetch Collection and Extract Data
    print("[INFO] Fetching vector store collection...")
    collection = vectorstore._collection
    if collection.count() == 0:
        print("[ERROR] Vector store is empty. Exiting...")
        exit()

    result = collection.get(include=["embeddings", "documents", "metadatas"])
    vectors = np.array(result["embeddings"])
    documents = result["documents"]
    doc_types = [metadata["doc_type"] for metadata in result["metadatas"]]

    # Step 4: Assign colors (simple approach; single category 'procedures')
    categories = ["procedures"]
    colors = [
        ["blue"][categories.index(t)]
        for t in doc_types
    ]

    # Step 5: 2D Visualization with t-SNE
    print("[INFO] Reducing dimensionality to 2D for visualization...")
    n_samples = len(vectors)
    perplexity = min(30, n_samples - 1)  # Perplexity must be < number of samples
    tsne_2d = TSNE(n_components=2, random_state=42, perplexity=perplexity)
    reduced_vectors_2d = tsne_2d.fit_transform(vectors)

    print("[INFO] Creating 2D scatter plot...")
    fig_2d = go.Figure(data=[go.Scatter(
        x=reduced_vectors_2d[:, 0],
        y=reduced_vectors_2d[:, 1],
        mode="markers",
        marker=dict(size=5, color=colors, opacity=0.8),
        text=[
            f"Type: {t}<br>Text: {d[:100]}..."
            for t, d in zip(doc_types, documents)
        ],
        hoverinfo="text"
    )])

    fig_2d.update_layout(
        title="2D Chroma Vector Store Visualization",
        xaxis_title="t-SNE Dimension 1",
        yaxis_title="t-SNE Dimension 2",
        width=800,
        height=600,
        margin=dict(r=20, b=10, l=10, t=40)
    )
    fig_2d.show()

    # Step 6: 3D Visualization with t-SNE
    print("[INFO] Reducing dimensionality to 3D for visualization...")
    tsne_3d = TSNE(n_components=3, random_state=42, perplexity=perplexity)
    reduced_vectors_3d = tsne_3d.fit_transform(vectors)

    print("[INFO] Creating 3D scatter plot...")
    fig_3d = go.Figure(data=[go.Scatter3d(
        x=reduced_vectors_3d[:, 0],
        y=reduced_vectors_3d[:, 1],
        z=reduced_vectors_3d[:, 2],
        mode="markers",
        marker=dict(size=5, color=colors, opacity=0.8),
        text=[
            f"Type: {t}<br>Text: {d[:100]}..."
            for t, d in zip(doc_types, documents)
        ],
        hoverinfo="text"
    )])

    fig_3d.update_layout(
        title="3D Chroma Vector Store Visualization",
        scene=dict(
            xaxis_title="t-SNE Dimension 1",
            yaxis_title="t-SNE Dimension 2",
            zaxis_title="t-SNE Dimension 3"
        ),
        width=900,
        height=700,
        margin=dict(r=20, b=10, l=10, t=40)
    )
    fig_3d.show()